In [42]:
from preprocessing_temp import prep_window_generator
import tensorflow as tf


import neptune
from neptune.integrations.tensorflow_keras import NeptuneCallback

import os


In [43]:
window_generator = prep_window_generator()


In [44]:
from typing import List


class Params:
    lstm_layer_count: int
    lstm_sizes: List[int]
    lstm_bidirectional: bool
    lstm_return_sequences: bool
    pooling_type: str
    kernel_sizes: List[int]
    dense_layer_count: int
    conv_layer_sizes: List[int]
    dropout: float
    input_width: int
    label_width: int
    batch_size: int
    loss_function: str

    def __init__(
        self,
        lstm_layer_count=1,
        lstm_sizes=[32],
        lstm_bidirectional=False,
        lstm_return_sequences=False,
        pooling_type="MaxPooling",
        kernel_sizes=[5, 5, 5],
        dense_layer_count=1,
        dense_layer_sizes=[512],
        conv_layer_sizes=[128, 256, 256],
        dropout=0.2,
        input_width=7 * 24,
        label_width=1 * 24,
        batch_size=32,
        loss_function="mse",
    ):
        self.lstm_layer_count = lstm_layer_count
        self.lstm_sizes = lstm_sizes
        self.lstm_bidirectional = lstm_bidirectional
        self.lstm_return_sequences = lstm_return_sequences
        self.pooling_type = pooling_type
        self.kernel_sizes = kernel_sizes
        self.dense_layer_count = dense_layer_count
        self.dense_layer_sizes = dense_layer_sizes
        self.conv_layer_sizes = conv_layer_sizes
        self.dropout = dropout
        self.input_width = input_width
        self.label_width = label_width
        self.batch_size = batch_size
        self.loss_function = loss_function

    def to_name(self, epochs: int):
        name = "cnn_"
        if self.lstm_bidirectional:
            name += "bi"
        name += "lstm_"
        name += str(int(self.input_width / 24)) + "days_history_"
        name += str(int(self.label_width / 24)) + "days_pred_"
        name += str(epochs) + "_epochs"
        if self.lstm_layer_count != 1:
            name += "_" + str(self.lstm_layer_count) + "lstm-layer"
            name += "_" + ",".join([str(i) for i in self.lstm_sizes])
        if self.lstm_return_sequences:
            name += "_ret-seq"
        if self.pooling_type != "MaxPooling":
            name += "_" + "pooling-" + self.pooling_type
        if self.dropout != 0.2:
            name += "_" + "dropout-0-" + str(int(self.dropout * 10))

        return name

    def to_dict(self):
        self_dict = self.__dict__
        for key, value in self_dict.items():
            if isinstance(value, list):
                self_dict[key] = str(value)
        return self_dict


In [45]:
def init_model(params: Params):
    num_features = 1
    init_kernel_size = params.kernel_sizes[0]
    cnn_lstm_model = tf.keras.models.Sequential()
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    cnn_lstm_model.add(tf.keras.layers.Normalization())
    cnn_lstm_model.add(tf.keras.layers.Lambda(lambda x: x[:, -init_kernel_size:, :]))
    # Shape => [batch, 1, conv_units]
    cnn_lstm_model.add(
        tf.keras.layers.Conv1D(
            params.conv_layer_sizes[0], activation="relu", kernel_size=params.kernel_sizes[0], padding="same"
        )
    )
    if params.pooling_type == "MaxPooling":
        cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
    else:
        cnn_lstm_model.add(tf.keras.layers.AveragePooling1D())
    cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
    cnn_lstm_model.add(
        tf.keras.layers.Conv1D(
            params.conv_layer_sizes[1], activation="relu", kernel_size=params.kernel_sizes[1], padding="same"
        )
    )
    if params.pooling_type == "MaxPooling":
        cnn_lstm_model.add(tf.keras.layers.MaxPooling1D())
    else:
        cnn_lstm_model.add(tf.keras.layers.AveragePooling1D())
    cnn_lstm_model.add(tf.keras.layers.BatchNormalization())  # TODO: useful to put it here?
    cnn_lstm_model.add(
        tf.keras.layers.Conv1D(
            params.conv_layer_sizes[2], activation="relu", kernel_size=params.kernel_sizes[2], padding="same"
        )
    )

    cnn_lstm_model.add(tf.keras.layers.Reshape((-1, 256)))
    # LSTM
    if params.lstm_layer_count == 2:
        lstm_layer1 = tf.keras.layers.LSTM(params.lstm_sizes[0], return_sequences=True)
        if params.lstm_bidirectional:
            cnn_lstm_model.add(tf.keras.layers.Bidirectional(lstm_layer1))
        else:
            cnn_lstm_model.add(lstm_layer1)

    lstm_layer2 = tf.keras.layers.LSTM(params.lstm_sizes[-1], return_sequences=params.lstm_return_sequences)
    if params.lstm_bidirectional:
        cnn_lstm_model.add(tf.keras.layers.Bidirectional(lstm_layer2))
    else:
        cnn_lstm_model.add(lstm_layer2)

    for i in range(params.dense_layer_count):
        cnn_lstm_model.add(tf.keras.layers.Dense(params.dense_layer_sizes[i]))
        cnn_lstm_model.add(tf.keras.layers.Dropout(params.dropout))

    cnn_lstm_model.add(
        tf.keras.layers.Dense(params.label_width * num_features, kernel_initializer=tf.initializers.zeros())
    )
    # Shape => [batch, out_steps, features] -> for each prediction step one neuron
    cnn_lstm_model.add(tf.keras.layers.Reshape([params.label_width, num_features]))
    return cnn_lstm_model


In [46]:
from typing import Dict


run_params: Dict[str, Params] = {}
run_params["base_run"] = Params()
run_params["lstm_size_64"] = Params(lstm_sizes=[64])
run_params["two_lstm_size_32"] = Params(lstm_layer_count=2, lstm_sizes=[32, 32])
run_params["two_lstm_size_64"] = Params(lstm_layer_count=2, lstm_sizes=[64, 64])
run_params["bidirectional_lstm"] = Params(lstm_bidirectional=True)
run_params["bidirectional_two_lstm"] = Params(lstm_bidirectional=True, lstm_layer_count=2, lstm_sizes=[32, 32])
run_params["average_pooling"] = Params(pooling_type="AveragePooling")
run_params["kernel_sizes_adjusted_to_hours"] = Params(kernel_sizes=[24, 12, 6])
run_params["two_dense_layer"] = Params(dense_layer_count=2, dense_layer_sizes=[512, 256])
run_params["dropout_0.4"] = Params(dropout=0.3)
run_params["dropout_0.4"] = Params(dropout=0.4)
run_params["dropout_0.5"] = Params(dropout=0.5)
run_params["dropout_0.6"] = Params(dropout=0.6)
run_params["loss_huber"] = Params(loss_function="huber")


In [47]:
for run_id, params in run_params.items():
    model = init_model(params)

    loss = tf.keras.losses.MeanSquaredError()
    if params.loss_function == "huber":
        loss = tf.keras.losses.huber
    model.compile(loss=loss, optimizer="Adam", metrics=[tf.keras.metrics.MeanAbsoluteError()])

    name = params.to_name(100)
    append_index = 0
    while os.path.exists("../models/" + name):
        append_index += 1
    if append_index != 0:
        name += "_" + str(append_index)

    run = neptune.init_run(
        project="data-mining-team2/initial-model-tests",
        api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4Mzg2ZWZmYi05YzRlLTQ3ODYtOWE1NC1mNDM4OTM1ZjNlOTkifQ==",
        custom_run_id=run_id,
        name=name,
        source_files=["./initial_model_tests.ipynb"],
    )

    run["parameters"] = params.to_dict()

    neptune_cbk = NeptuneCallback(run=run, base_namespace="training")

    try:
        for epoch_count_factor in range(1, 5):
            epoch_count = 50 * epoch_count_factor
            history = model.fit(
                window_generator.train,
                epochs=50,
                validation_data=window_generator.val,
                callbacks=[neptune_cbk],
            )

            eval_metrics = model.evaluate(window_generator.test, verbose=0)
            for j, metric in enumerate(eval_metrics):
                run[f"eval/epoch_{epoch_count}/{model.metrics_names[j]}"] = metric

            name = params.to_name(epoch_count)
            temp_name = name
            append_index = 0
            while os.path.exists("../models/" + temp_name):
                append_index += 1
                temp_name = name + "_" + str(append_index)
            if append_index != 0:
                name += "_" + str(append_index)

            model.save("../models/" + name, include_optimizer=False)

            run["model_names/" + str(epoch_count) + "epochs"] = name
    except:
        pass
    run.stop()


https://app.neptune.ai/data-mining-team2/initial-model-tests/e/IMT-10
Epoch 1/5
728/728 [==============================] - 15s 17ms/step - loss: 393.6563 - mean_absolute_error: 13.0762 - val_loss: 323.5807 - val_mean_absolute_error: 12.0086
Epoch 2/5
728/728 [==============================] - 12s 16ms/step - loss: 329.2359 - mean_absolute_error: 11.9731 - val_loss: 320.6897 - val_mean_absolute_error: 11.6369
Epoch 3/5
728/728 [==============================] - 12s 16ms/step - loss: 319.5548 - mean_absolute_error: 11.7887 - val_loss: 315.7220 - val_mean_absolute_error: 12.2315
Epoch 4/5
728/728 [==============================] - 12s 16ms/step - loss: 313.5499 - mean_absolute_error: 11.6578 - val_loss: 308.0201 - val_mean_absolute_error: 11.2329
Epoch 5/5
728/728 [==============================] - 13s 17ms/step - loss: 306.0692 - mean_absolute_error: 11.4900 - val_loss: 301.0569 - val_mean_absolute_error: 11.2925


INFO:tensorflow:Assets written to: ../models/cnn_lstm_24days_history_3days_pred_50_epochs\assets


INFO:tensorflow:Assets written to: ../models/cnn_lstm_24days_history_3days_pred_50_epochs\assets


Epoch 1/5
728/728 [==============================] - 10s 13ms/step - loss: 301.7361 - mean_absolute_error: 11.4438 - val_loss: 368.4911 - val_mean_absolute_error: 13.5272
Epoch 2/5
728/728 [==============================] - 10s 13ms/step - loss: 298.3039 - mean_absolute_error: 11.3692 - val_loss: 308.7127 - val_mean_absolute_error: 11.6110
Epoch 3/5
728/728 [==============================] - 10s 14ms/step - loss: 293.2456 - mean_absolute_error: 11.2693 - val_loss: 302.9725 - val_mean_absolute_error: 11.1262
Epoch 4/5
728/728 [==============================] - 9s 12ms/step - loss: 286.9782 - mean_absolute_error: 11.1419 - val_loss: 285.7923 - val_mean_absolute_error: 11.2618
Epoch 5/5
728/728 [==============================] - 9s 12ms/step - loss: 276.7703 - mean_absolute_error: 10.9067 - val_loss: 281.5990 - val_mean_absolute_error: 10.9524


INFO:tensorflow:Assets written to: ../models/cnn_lstm_24days_history_3days_pred_100_epochs\assets


...

INFO:tensorflow:Assets written to: ../models/cnn_lstm_24days_history_3days_pred_150_epochs\assets


Epoch 1/5
728/728 [==============================] - 12s 17ms/step - loss: 246.6264 - mean_absolute_error: 10.2622 - val_loss: 293.9261 - val_mean_absolute_error: 11.0571
Epoch 2/5
728/728 [==============================] - 12s 17ms/step - loss: 241.9731 - mean_absolute_error: 10.1712 - val_loss: 256.5767 - val_mean_absolute_error: 10.4647
Epoch 3/5
728/728 [==============================] - 12s 17ms/step - loss: 235.2334 - mean_absolute_error: 10.0363 - val_loss: 262.5401 - val_mean_absolute_error: 10.4293
Epoch 4/5
728/728 [==============================] - 12s 17ms/step - loss: 231.3583 - mean_absolute_error: 9.9663 - val_loss: 249.4443 - val_mean_absolute_error: 10.1677
Epoch 5/5
728/728 [==============================] - 12s 17ms/step - loss: 223.0656 - mean_absolute_error: 9.8070 - val_loss: 253.5987 - val_mean_absolute_error: 10.1698


INFO:tensorflow:Assets written to: ../models/cnn_lstm_24days_history_3days_pred_200_epochs\assets


INFO:tensorflow:Assets written to: ../models/cnn_lstm_24days_history_3days_pred_200_epochs\assets


Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 3 operations to synchronize with Neptune. Do not kill this process.
All 3 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/data-mining-team2/initial-model-tests/e/IMT-10/metadata
